# Rigid Body Simulator

Credit to Professor Bo Zhu, on whose COSC 89 Course Notes the following code is based on

## Geometry in space

In [72]:
import taichi as ti
import numpy as np

ti.init()

# variables and fields to keep track of
num_particles = 1
dt = 1e-3
substeps = 10
mass = 1

# Inertial tensor
I = 10
# position - center of mass
x = ti.Vector.field(n=2, dtype=ti.f32, shape=num_particles)
# x = ti.Vector([0.5, 0.5])
# vertices - relative to center of mass -- will have to change if more than one solid
vertices_0 = ti.Vector.field(n=2, dtype=ti.f32, shape=num_particles*3)
rotated_vertices = ti.Vector.field(n=2, dtype=ti.f32, shape=num_particles*3)
# force
force = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles)
# torque
torque = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles)
# velocity
v = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles)
# orientation
theta = ti.field(dtype=ti.f32, shape=num_particles)
# angular velocity
w = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles)

@ti.kernel
def init_pos():
    x[0] = ti.Vector([0.5, 0.5])
    # equilateral triangle
    vertices_0[0] = ti.Vector([0, 0.1])
    vertices_0[1] = ti.Vector([-0.0866, -0.05])
    vertices_0[2] = ti.Vector([0.0866, -0.05])
    rotated_vertices[0] = ti.Vector([0, 0.1])
    rotated_vertices[1] = ti.Vector([-0.0866, -0.05])
    rotated_vertices[2] = ti.Vector([0.0866, -0.05])
    # initialize velocities
    v[0] = ti.Vector([0, 0, 0])
    # initialize force
    force[0] = ti.Vector([0,0,0])
    

@ti.func
def calculate_force():
    # gravity
    force = force - ti.Vector([0, -2, 0])

@ti.func
def rotate():
    pass

@ti.func
def calculate_torque():
    pass

@ti.kernel
def substep():
    calculate_force()
    calculate_torque()
    # update velocity
    v[0] = v[0] + dt * force[0] / mass
    # update angular velocity
    w[0] = w[0] + dt * torque[0] / I
    # update position
    x[0] = x[0] + dt * v[0]
    # update orientation
    theta[0] = theta[0] * dt * w[0]
    # update vertices
    rotate()

def main():
    gui = ti.GUI('Rigid Body', (600, 600))
    # initialize vertices
    init_pos()
    
    while gui.running:
        for e in gui.get_events(ti.GUI.PRESS):
            pass
            
        # Update for a time step
        for step in range(substeps):
            substep()

        gui.triangle(a=x[0]+rotated_vertices[0], b=x[0]+rotated_vertices[1], c=x[0]+rotated_vertices[2], color=0xED553B)
        
        gui.show()
    
if __name__ == '__main__':
    main()

[Taichi] Starting on arch=x64


Code from stackoverflow on finding the inertial tensor using triangles

https://stackoverflow.com/questions/41592034/computing-tensor-of-inertia-in-2d

## Particle Sand

In [137]:
import taichi as ti

ti.init()

max_particles = 100
num_particles = ti.field(dtype=ti.i32, shape=())
dt = 1e-3
substeps = 10

ks = 2e2
kd = 0.5e1

x = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
v = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
f = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
r = ti.field(dtype=ti.f32, shape=max_particles)
m = ti.field(dtype=ti.f32, shape=max_particles)


@ti.kernel
def substep():
    # find force
    for i in ti.ndrange(num_particles[None]):
        f[i] = [0,-2] # reset force, since we're calculating it from scratch each time
        # check for collisions
        # boundary check
        phi = x[i][1] - r[i]/640
        # phi = (ti.Vector([x[i][0], 0]) - x[i]).norm() - r[i]/640
        if phi < 0: # lower boundary
            f[i] += ks * (phi) * ti.Vector([0,-1])
            f[i] += kd * (ti.Vector([0,0]) - v[i]).dot(ti.Vector([0,-1])) * ti.Vector([0, -1])
            ###
            # How do I make it so the resting state isn't slightly submerged
            ###
        # other particles
        for j in range(i, num_particles[None]):
            # check for collision
            phi = (x[i] - x[j]).norm() - r[i]/640 - r[j]/640
            if phi < 0:
                force = ks * (phi) * (x[j] - x[i]).normalized()
                f[i] += force
                f[j] -= force
#                 f[i] += kd * (ti.Vector([0,0]) - v[i]).dot(ti.Vector([0,-1])) * ti.Vector([0, -1])
        
    for i in range(num_particles[None]):
        # update v
        v[i] = v[i] + dt * f[i] / m[i]
        # update x
        x[i] = x[i] + dt * v[i]

        
@ti.kernel
def new_particle(pos_x: ti.f32, pos_y: ti.f32):
    particle_id = num_particles[None]
    if particle_id < max_particles:
        x[particle_id] = [pos_x, pos_y]
        v[particle_id] = [0, 0]
        m[particle_id] = 1
        r[particle_id] = 10
        num_particles[None] += 1
        

def main():
    gui = ti.GUI('Particle Sand', (640, 640))
    
    new_particle(0.3, 0.5)
    
    while gui.running:
        for e in gui.get_events(ti.GUI.PRESS):
            if e.key == ti.GUI.UP:
                ks *= 1.1
            elif e.key == ti.GUI.DOWN:
                ks /= 1.1
            elif e.key == ti.GUI.LMB:
                new_particle(e.pos[0], e.pos[1])
            
        # Move stuff
        for step in range(substeps):
            substep()
             
        # Draw particles
        X = x.to_numpy()
        for i in range(num_particles[None]):
            gui.circle(pos=X[i], radius = r[i])

        # show the current values
        gui.text(
            content='Click anywhere on screen to add a new mass',
            pos=(0,0.99),
            color=0xffffff)
        
        gui.show()
    
if __name__ == '__main__':
    main()

[Taichi] Starting on arch=x64


[Taichi] Starting on arch=x64
